In [1]:
import numpy as np

# .npy 파일 경로
file_path = './content/tracked_data.npy'  # 파일 경로를 적절히 설정하세요.

# .npy 파일 불러오기
data = np.load(file_path, allow_pickle=True)

fps=15

In [2]:
# 차량 등장 횟수 계산
vehicle_count = {}

for entry in data:
    for vehicle in entry.keys():
        if vehicle in vehicle_count:
            vehicle_count[vehicle] += 1
        else:
            vehicle_count[vehicle] = 1


filtered_data = []
for entry in data:
    filtered_entry = {vehicle: entry[vehicle] for vehicle in entry.keys() if vehicle_count[vehicle] > 2*fps}
    if filtered_entry:  # 필터링된 결과가 비어있지 않으면 추가
        filtered_data.append(filtered_entry)

# 결과 출력
print("필터링된 데이터:")
for entry in filtered_data:
    print(entry)

필터링된 데이터:
{'vehicle0': array([1028,  465, 1077,  502]), 'vehicle1': array([ 916,  460, 1003,  531])}
{'vehicle0': array([1028.6763 ,  465.03943, 1077.637  ,  502.43475], dtype=float32), 'vehicle1': array([ 916.2885 ,  460.58807, 1002.2643 ,  531.17596], dtype=float32)}
{'vehicle0': array([1029.395  ,  465.68445, 1078.5945 ,  503.4257 ], dtype=float32), 'vehicle1': array([ 916.66943,  461.46814, 1002.0926 ,  531.51636], dtype=float32)}
{'vehicle0': array([1030.9377 ,  465.91434, 1080.12   ,  503.62146], dtype=float32), 'vehicle1': array([ 917.3276 ,  461.63507, 1002.9005 ,  531.063  ], dtype=float32)}
{'vehicle0': array([1030.9658 ,  465.5088 , 1080.6576 ,  503.17758], dtype=float32), 'vehicle1': array([ 917.4484 ,  461.06995, 1002.6976 ,  530.53394], dtype=float32)}
{'vehicle0': array([1032.7042,  466.3739, 1082.3601,  504.305 ], dtype=float32), 'vehicle1': array([ 917.9553 ,  462.24588, 1003.60596,  531.18726], dtype=float32)}
{'vehicle0': array([1034.2523 ,  465.6954 , 1083.9238 ,  5

In [3]:
vehicle_count = {}

for entry in filtered_data:
    for vehicle in entry.keys():
        if vehicle in vehicle_count:
            vehicle_count[vehicle] += 1
        else:
            vehicle_count[vehicle] = 1

# 결과 출력
for vehicle, count in vehicle_count.items():
    print(f"{vehicle}가 {count}번 등장했습니다.")

vehicle0가 132번 등장했습니다.
vehicle1가 82번 등장했습니다.
vehicle8가 45번 등장했습니다.


In [4]:
np.save('tracked_data.npy', filtered_data)

In [5]:
def calculate_area(bbox):
    x_min, y_min, x_max, y_max = bbox
    return (x_max - x_min) * (y_max - y_min)

def intersect(bbox1, bbox2):
    x1_min, y1_min, x1_max, y1_max = bbox1
    x2_min, y2_min, x2_max, y2_max = bbox2

    # 겹치는 부분의 좌표 계산
    intersect_x_min = max(x1_min, x2_min)
    intersect_y_min = max(y1_min, y2_min)
    intersect_x_max = min(x1_max, x2_max)
    intersect_y_max = min(y1_max, y2_max)

    if intersect_x_max > intersect_x_min and intersect_y_max > intersect_y_min:
        return (intersect_x_min, intersect_y_min, intersect_x_max, intersect_y_max)
    return None  # 겹치지 않음

def subtract_intersection(bbox1, intersection):
    if intersection is None:
        return [bbox1]  # 겹치지 않으면 원래 bbox를 리스트로 반환

    x1_min, y1_min, x1_max, y1_max = bbox1
    ix_min, iy_min, ix_max, iy_max = intersection

    new_boxes = []

    # 겹치지 않는 부분 추가
    if y1_min < iy_min:
        new_boxes.append([x1_min, y1_min, x1_max, iy_min])  # 아래쪽 부분

    if y1_max > iy_max:
        new_boxes.append([x1_min, iy_max, x1_max, y1_max])  # 위쪽 부분

    if x1_min < ix_min:
        new_boxes.append([x1_min, max(y1_min, iy_min), ix_min, min(y1_max, iy_max)])  # 왼쪽 부분

    if x1_max > ix_max:
        new_boxes.append([ix_max, max(y1_min, iy_min), x1_max, min(y1_max, iy_max)])  # 오른쪽 부분

    return new_boxes

def select_largest_non_overlapping(bounding_boxes):
    remaining_boxes = bounding_boxes.copy()
    selected_boxes = []

    while remaining_boxes:
        # 가장 큰 면적의 bounding box 선택
        largest_box_key = max(remaining_boxes, key=lambda k: calculate_area(remaining_boxes[k]))
        largest_box = remaining_boxes.pop(largest_box_key)

        # 선택된 박스를 결과에 추가
        selected_boxes.append((largest_box_key, largest_box))

        # 나머지 bounding box에서 겹치는 부분 제거
        new_remaining_boxes = {}
        for key in list(remaining_boxes.keys()):
            intersection = intersect(largest_box, remaining_boxes[key])
            if intersection:
                # 겹치는 부분이 있을 경우 남은 부분을 추가
                new_boxes = subtract_intersection(remaining_boxes[key], intersection)
                for idx, new_box in enumerate(new_boxes):
                    # 기존 key에 idx를 추가해 고유하게 만듦
                    new_remaining_boxes[f"{key}_{idx}"] = new_box
            else:
                # 겹치지 않으면 그대로 추가
                new_remaining_boxes[key] = remaining_boxes[key]

        # 새로 남은 bounding box를 업데이트
        remaining_boxes = new_remaining_boxes

    # 최종 결과 생성: 각 vehicle에 대해 하나의 bounding box만 남기기
    final_boxes = {}
    for key, box in selected_boxes:
        vehicle_id = key.split('_')[0]  # 원래 vehicle ID를 복원
        if vehicle_id not in final_boxes:
            final_boxes[vehicle_id] = box
        else:
            # 이미 존재하는 vehicle이라면 겹치는 부분을 조정
            existing_box = final_boxes[vehicle_id]
            intersection = intersect(existing_box, box)
            if intersection:
                # 겹치는 부분이 있다면 겹치지 않는 부분 추가
                new_boxes = subtract_intersection(box, intersection)
                for nb in new_boxes:
                    final_boxes[vehicle_id] = nb

    return list(final_boxes.items())


# 예시 bounding_boxes
result=[]
for i in range(len(filtered_data)):
    temp=select_largest_non_overlapping(filtered_data[i])
    temp2={}
    for t in temp:
        temp2[t[0]]=np.array(t[1])
    result.append(temp2)

print(result[20])

{'vehicle1': array([ 917.75995,  458.68408, 1001.1304 ,  527.2299 ], dtype=float32), 'vehicle0': array([1055.9281 ,  463.01816, 1107.9072 ,  504.46024], dtype=float32), 'vehicle8': array([914.7747 , 442.18805, 962.3818 , 458.68408], dtype=float32)}


In [7]:
np.save('crop_tracked_data.npy', result)

In [8]:
print(filtered_data[66]) ##기존
print(result[66])  ## 변경본

{'vehicle0': array([1035.8262 ,  454.22018, 1144.6208 ,  511.55853], dtype=float32), 'vehicle1': array([907.63794, 450.06702, 982.66364, 508.17413], dtype=float32)}
{'vehicle0': array([1035.8262 ,  454.22018, 1144.6208 ,  511.55853], dtype=float32), 'vehicle1': array([907.63794, 450.06702, 982.66364, 508.17413], dtype=float32)}
